In [43]:
import numpy as np

class Environment:
    def position(self):
        assert 0, "position not implemented"
        
class Simulation(Environment):
    def __init__(self):
        self.state = np.zeros(13) # The state vector [jx, jy, jz, then 4 quaternions, x, y, z, xdot, ydot, zdot]
        
    def propagate(self, initial_state, timestep):
        a = 1
        # This should call orbit propagator

class PhysicalSat:
    def __init__(self, environment):
        self.angular_velocity = np.array([0,0,0])
        self.battery_voltage = 10
        self.battery_beacon_threshold = 3.6 #Volts
        self.battery_tumble_threshold = 3.6 #Volts
        self.environment = environment
        
    def getBatteryVoltage(self):
        return self.battery_voltage
    
    def getAngularVelocity(self):
        # How are we going to estimate how the vechile perceives it's actual state?
        return self.angular_velocity

In [47]:
# Based off state machine found here: https://python-3-patterns-idioms-test.readthedocs.io/en/latest/StateMachine.html

class State:
    def run(self):
        assert 0, "run not implemented"
    def next(self, input):
        assert 0, "next not implemented"

class StateMachine:
    def __init__(self, initialState, hardware):
        self.currentState = initialState
        self.hardware = hardware
        self.currentState.run(self.hardware)
        
    # Template method:
    def runAll(self, inputs):
        for i in inputs:
            #print(i)
            self.currentState = self.currentState.next(self.hardware)
            self.currentState.run(self.hardware)
            
    def runNext(self, inputs):
        self.currentState = self.currentState.next(inputs, self.hardware)
        self.currentState.run(self.hardware)

In [53]:
class Hold(State):
    def run(self, hardware):
        print("Hold: Initial Boot and hold")

    def next(self, hardware):
        return PandaSat.deploy_antenna

class DeployAntenna(State):
    def run(self, hardware):
        print("Deploying Antenna")

    def next(self, hardware):
        return PandaSat.tumble_check
    
class TumbleCheck(State):
    def run(self, hardware):
        print('Checking the tumbling rate')
        
    def next(self, hardware):
        #tumbling
        if self.tumble_compare(hardware):
            return PandaSat.battery_tumble_check()
        return PandaSat.sleep
        
    def tumble_compare(self, hardware):
        # Implements the logic of the tumble check
        # Return True for tumbling too fast or False for not tumbling above the threshold
        threshold_value = 1 #1 rad/s?
        rate_boolean = hardware.getAngularVelocity() > threshold_value
        if np.sum(rate_boolean) >= 1:
            return True
        return False
        
class Sleep(State):
    def run(self, hardware):
        print('Sleep Mode - 1 min')
        
    def next(self, hardware):
        return PandaSat.battery_beacon_check
    
class BatteryTumbleCheck(State):
    def run(self, hardware):
        print('Checking battery voltage before 1 min of detumbling')
        
    def next(self, hardware):
        battery_voltage = hardware.getBatteryVoltage()
        battery_tumble_threshold = hardware.battery_tumble_threshold
        if battery_voltage < battery_tumble_threshold:
            return PandaSat.sleep
        return PandaSat.detumble
    
class Detumble(State):
    def run(self, hardware):
        print('Detumbling the spacecraft for 1 min')
        
    def next(self, hardware):
        return PandaSat.battery_beacon_check
    
class BatteryBeaconCheck(State):
    def run(self, hardware):
        print('Checking battery voltage before sending beacon')
        
    def next(self, hardware):
        battery_voltage = hardware.getBatteryVoltage()
        battery_beacon_threshold = hardware.battery_beacon_threshold
        if battery_voltage < battery_beacon_threshold:
            return PandaSat.sleep
        return PandaSat.beacon
    
class Beacon(State):
    def run(self, hardware):
        print('Sending health data beacon signal')
    
    def next(self, hardware):
        return PandaSat.listen

class Listen(State):
    def run(self, hardware):
        print('Listening for ground signal for ~15s')
        
    def next(self, hardware):
        return PandaSat.uplink_check
    
class UplinkCheck(State):
    def run(self, hardware):
        print('Checking if an uplink was made')

class PandaSat(StateMachine):
    def __init__(self, hardware):
        # Initial state
        StateMachine.__init__(self, PandaSat.hold, hardware)

# Static variable initialization:
PandaSat.hold = Hold()
PandaSat.deploy_antenna = DeployAntenna()
PandaSat.tumble_check = TumbleCheck()
PandaSat.sleep = Sleep()
PandaSat.battery_tumble_check = BatteryTumbleCheck()
PandaSat.detumble = Detumble()
PandaSat.battery_beacon_check = BatteryBeaconCheck()
PandaSat.beacon = Beacon()
PandaSat.listen = Listen()
PandaSat.uplink_check = UplinkCheck()

In [54]:
orbit_sim = Simulation()
hardware = PhysicalSat(orbit_sim)
inputs = (None,None,None,None,None, None, None)
ps = PandaSat(hardware)
ps.runAll(inputs)



Hold: Initial Boot and hold
Deploying Antenna
Checking the tumbling rate
Sleep Mode - 1 min
Checking battery voltage before sending beacon
Sending health data beacon signal
Listening for ground signal for ~15s
Checking if an uplink was made
